In [36]:
from bs4 import BeautifulSoup as bs
import requests


from shutil import copyfile

import js2py
from utils import getHash

from tqdm import tqdm_notebook as tqdm

In [2]:
def get_tasks_from_theme(theme_url):
    skip = 0
    result = []
    while True:
        res = requests.post(theme_url,data = {'ajax':1,'skip':skip})
        soup = bs(res.content, 'lxml')
        tasks = soup.find_all('div', {'class': 'prob_view'})
        if len(tasks) == 0:
            break
        result += tasks
        skip += 5
        
    return result

In [12]:
def get_hash_from_task(task):
    task_content = task.findChild().findChild().findChild().nextSibling()[0]
    
    task_img_srcs = ''.join([img['src'] for img in task_content.find_all('img')])
    task_text = task_content.text
    text_merged = task_img_srcs + task_text
    text_merged = text_merged.replace('\xa0', '').replace(' ', '')
    
    text_for_hash = ''.join(sorted(text_merged))
    hashed = getHash(text_for_hash)
    
    return hashed

In [13]:
def get_task_number(task):
    return task.findChild().findChild().findChild().find('a').text

In [30]:
# remove math- in for all subjects
theme_links = [a for a, b in [i.split('|||') for i in open('themes_list.txt','r').read().split('\n')] if b !='0' and 'math-' in a]

In [32]:
inners_dataset = []
for url in tqdm(theme_links):
    tasks = get_tasks_from_theme(url)
    inners = []
    for task in tasks:
        task_hash = get_hash_from_task(task)
        if task_hash != 0:
            task_number = get_task_number(task)
            inners.append([task_hash, task_number])
    inners_dataset += inners

A Jupyter Widget

In [46]:
inners_dataset = dict(inners_dataset)

In [47]:
import json
with open('hash_id.json', 'w') as fp:
    json.dump(inners_dataset, fp)
    
copyfile('hash_id.json', './extention/hash_id.json')

'./extention/hash_id.json'

In [ ]:
# import json
# with open('all.json', 'w') as fp:
#     json.dump(inners_dataset, fp)

In [ ]:
# with open('hash_id_dict.json', 'r') as f:
#     distros_dict = json.load(f)

In [ ]:
# open('dict_in_txt.txt', 'w').write(str(distros_dict))